In [ ]:
from google.colab import drive
drive.mount('/content/drive/')

Mounted at /content/drive/


In [ ]:
!pip install torch torchvision torchaudio transformers sentencepiece --quiet
!pip install google-generativeai

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 99.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 81.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 49.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 14.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 6.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 56.3 MB/s eta 0:00:00


In [ ]:
import os
import pandas as pd
from tqdm import tqdm
from google.colab import userdata
from google import genai
import sys

os.environ["GOOGLE_API_KEY"] = userdata.get('gemini_key')

pd.set_option('display.max_rows', 1000)

In [ ]:
base_folder_path = '/content/drive/MyDrive/nlp/final'

os.listdir(base_folder_path)

['ans_gen.ipynb',
 'QA_1001-1100.csv',
 'dataset-pdf',
 'GEMINI V.1',
 'eval',
 'prediction_with_clean_ds',
 'baseline',
 'attempts',
 'test_answer_list.csv',
 'answer_sampling_question.csv',
 'cos_sim_normalization[0-1]',
 'bert_score_F1[0-1]',
 'generated answer',
 'eval_content.ipynb',
 'try_to_eval_structure.ipynb',
 'similarity_scores_by_model.gsheet']

In [ ]:
src_folder = os.path.join(base_folder_path, 'dataset-pdf')
sum_folder = os.path.join(base_folder_path, 'GEMINI V.1/slide.md file V.1')
lora_folder = os.path.join(base_folder_path, 'prediction_with_clean_ds')
baseline_folder = os.path.join(base_folder_path, 'baseline')

print("Files in dataset-pdf:", os.listdir(src_folder)[:5])
print("Files in eval:", os.listdir(sum_folder)[:5])
print("Files in prediction_with_clean_ds:", os.listdir(lora_folder)[:5])
print("Files in baseline:", os.listdir(baseline_folder)[:5])

Files in dataset-pdf: ['90', '95', '88', '94', '89']
Files in eval: ['slide_1009.md', 'slide_1003.md', 'slide_1021.md', 'slide_1027.md', 'slide_1020.md']
Files in prediction_with_clean_ds: ['slide_0.md', 'slide_1.md', 'slide_2.md', 'slide_3.md', 'slide_4.md']
Files in baseline: ['slide_1002.md', 'slide_1003.md', 'slide_1004.md', 'slide_1005.md', 'slide_1006.md']


In [ ]:
eval_src = [os.path.join(src_folder, str(i)) for i in range(1001,1101)]
eval_sum = [os.path.join(sum_folder, f'slide_{str(i)}.md') for i in range(1001,1101)]
eval_lora = [os.path.join(lora_folder, f'slide_{str(i)}.md') for i in range(0,100)]
eval_baseline = [os.path.join(baseline_folder, f'slide_{str(i)}.md') for i in range(1001,1101)]

print(eval_src)
print(eval_sum)
print(eval_lora)
print(eval_baseline)

['/content/drive/MyDrive/nlp/final/dataset-pdf/1001', '/content/drive/MyDrive/nlp/final/dataset-pdf/1002', '/content/drive/MyDrive/nlp/final/dataset-pdf/1003', '/content/drive/MyDrive/nlp/final/dataset-pdf/1004', '/content/drive/MyDrive/nlp/final/dataset-pdf/1005', '/content/drive/MyDrive/nlp/final/dataset-pdf/1006', '/content/drive/MyDrive/nlp/final/dataset-pdf/1007', '/content/drive/MyDrive/nlp/final/dataset-pdf/1008', '/content/drive/MyDrive/nlp/final/dataset-pdf/1009', '/content/drive/MyDrive/nlp/final/dataset-pdf/1010', '/content/drive/MyDrive/nlp/final/dataset-pdf/1011', '/content/drive/MyDrive/nlp/final/dataset-pdf/1012', '/content/drive/MyDrive/nlp/final/dataset-pdf/1013', '/content/drive/MyDrive/nlp/final/dataset-pdf/1014', '/content/drive/MyDrive/nlp/final/dataset-pdf/1015', '/content/drive/MyDrive/nlp/final/dataset-pdf/1016', '/content/drive/MyDrive/nlp/final/dataset-pdf/1017', '/content/drive/MyDrive/nlp/final/dataset-pdf/1018', '/content/drive/MyDrive/nlp/final/dataset-pdf

In [ ]:
os.listdir(eval_src[0])

['figures_and_tables.txt',
 'extracted_text.txt',
 'paper.tei.xml',
 'input_prompt.txt',
 'paper.pdf',
 'extracted_text_with_tag.txt',
 'slide.md',
 '1212.windowed_summarunner_scores.txt',
 'slide.xml',
 'slide.pdf',
 'images']

In [ ]:
with open(os.path.join(eval_src[0], 'slide.md'), 'r', encoding='utf-8') as f:
  content = f.read()
  test_src = content

print(content)

---
theme: default
title: Fast and Secure Laptop Backups with Encrypted De-duplication
class: text-center

---

# Fast and Secure Laptop Backups with Encrypted De-duplication
Paul Anderson, Le Zhang

---

# Laptop Backup Options

- External Hard Drive
- No offsite storage ? What if I have a break-in? Or there is a fire?
- I need a very large capacity to handle archival storage as well ...

---

# Laptop Backup Options

- Recordable CD/DVD
- I have to make multiple copies if I want offsite storage ...
- DVDs are only small - I can only backups subsets of files ...

---

# Laptop Backup Options

- Cloud Storage
- Broadband upload speeds are slow - 30 DAYS to upload 300Gb to cloud storage is typical ...
- Often, there is a transfer cost as well as a storage cost ...

---

# Laptop Backup Options

- External Hard Drive
- Recordable CD/DVD
- Cloud Storage

---

# What do people do?

- Store no vital data Regular full backups Partial backups Keep copy on University machine Dont do backups Do

In [ ]:
with open(eval_sum[0], 'r', encoding='utf-8') as f:
  content = f.read()
  test_sum = content

print(content)

---
aspectRatio: 16/9
canvasWidth: 1200
layout: cover
---

# Fast and Secure Laptop Backups with Encrypted De-duplication

<div class="text-xl mt-4">

Paul Anderson & Le Zhang
<br>
University of Edinburgh

</div>
---
layout: default
---

# Abstract & Keywords

<div class="text-2xl space-y-4">

*   **Problem**: Conventional backup solutions are ill-suited for laptops, which have poor/intermittent connectivity and are vulnerable to theft/hardware failure.
*   **Solution**: An algorithm leveraging common data between users to speed up backups and reduce storage, supporting client-end per-user encryption.
*   **Unique Feature**: Immediate detection of common subtrees, avoiding queries for every file.
*   **Implementation**: Prototype for Apple OS X, analysis of potential effectiveness with real user data.
*   **Keywords**: backup, de-duplication, encryption, cloud computing.

</div>

---
layout: default
---

# Introduction: The Backup Challenge

<div class="text-2xl space-y-4">

*   Data b

In [ ]:
with open(eval_lora[0], 'r', encoding='utf-8') as f:
  content = f.read()
  test_lora = content

print(content)

---
theme: default
title: Fast and Secure Laptop Backups with Encrypted De-duplication
class: text-center

---
# Fast and Secure Laptop Backups with Encrypted De-duplication
Paul Anderson, Le Zhang

---
# Backup Statistics

- 24 Mac laptops
- 2,990,133 files (including 326,416 directories)
- Total disk space used: 114,868 GB
- 5.6 TB shared
- 5.2 MB average file size
- 50 MB largest file

---
# Laptop Backup Problems

- **Discontinuously Performed Backups:** Most backup solutions are not suitable for the "move-work" computing paradigm.
- **Nightmare Scenarios:** Laptop stolen or lost. Data loss.
- **Mixing Personal and Corporate Data:** Inadvertent storage of sensitive information in unsecured cloud services.

---
# Laptop Backup Solutions

- **Incomplete and Manual Solutions:** Ad hoc backup strategies, backup to CD/DVDs, USB drives.
- **Modern Backup Solutions:** Cloud, automated, incremental.
- **Potential Problems:** Backup times too high, unreliable network connection required.

-

In [ ]:
with open(eval_baseline[0], 'r', encoding='utf-8') as f:
  content = f.read()
  test_baseline = content

print(content)

# Fast and Secure Laptop Backups with Encrypted De-duplication

**Authors:** Paul Anderson, Le Zhang

**Abstract:**
Many people now store large quantities of personal and corporate data on laptops or home computers. These often have poor or intermittent connectivity, and are vulnerable to theft or hardware failure. Conventional backup solutions are not well suited to this environment, and backup regimes are frequently inadequate. This paper describes an algorithm which takes advantage of the data which is common between users to increase the speed of backups, and reduce the storage requirements. This algorithm supports client-end per-user encryption which is necessary for confidential personal data. It also supports a unique feature which allows immediate detection of common subtrees, avoiding the need to query the backup system for every file. We describe a prototype implementation of this algorithm for Apple OS X, and present an analysis of the potential effectiveness, using real dat

In [ ]:
src_data = []
id = 1001

for i in tqdm(range(len(eval_src))):

  src_md_path = os.path.join(eval_src[i], 'slide.md')
  sum_md_path = eval_sum[i]
  lora_md_path = eval_lora[i]
  baseline_md_path = eval_baseline[i]

  if os.path.exists(src_md_path):
    with open(src_md_path, 'r', encoding='utf-8') as f_src:
      source = f_src.read()
  else:
    source = None

  if os.path.exists(sum_md_path):
    with open(sum_md_path, 'r', encoding='utf-8') as f_sum:
      summary = f_sum.read()
  else:
    summary = None

  if os.path.exists(lora_md_path):
    with open(lora_md_path, 'r', encoding='utf-8') as f_lora:
      lora = f_lora.read()
  else:
    lora = None

  if os.path.exists(baseline_md_path):
    with open(baseline_md_path, 'r', encoding='utf-8') as f_baseline:
      baseline = f_baseline.read()
  else:
    baseline = None

  src_data.append({
    'paper_id': id,
    'source': source,
    'summary': summary,
    'lora': lora,
    'baseline': baseline
  })

  id += 1

data_df = pd.DataFrame(src_data)

  1%|          | 1/100 [00:04<07:29,  4.54s/it]


KeyboardInterrupt: 

In [ ]:
data_df

In [ ]:
question_df = pd.read_csv(os.path.join(base_folder_path, "QA_1001-1100.csv"))

In [ ]:
question_df

,paper_id,section,question,answer
0,1001,Title,What is the title of the paper?,Fast and Secure Laptop Backups with Encrypted ...
1,1001,Abstract:,What type of data does the algorithm take adva...,common
2,1001,Section 1: Introduction,How many TB of data do people have at home?,1
3,1001,Section 1: Introduction,How long did it take to get all of the data co...,three months
4,1001,Section 1: Introduction,What type of laptops are used to backup?,personal
...,...,...,...,...
6418,1100,Section 7: Conclusion,How much overhead do our unoptimized prototype...,10-15%
6419,1100,Section 7: Conclusion,How does DBTaint compare to Web applications?,transparent
6420,1100,Section 7: Conclusion,What can DBTaint identify due to incomplete sa...,security vulnerabilities
6421,1100,Section 7: Conclusion,What does DBTaint require to the database?,SQL-level additions


In [ ]:
question_df = question_df[
    (question_df['section'] == 'Title') |
    (~(question_df['question'].str.contains(r'\bwhy\b', case=False, regex=True)) &
    ~(question_df['answer'].str.contains(r'\bsection\b', case=False, regex=True, na=False)))
]

In [ ]:
question_df[question_df['section'] == 'Title']

,paper_id,section,question,answer
0,1001,Title,What is the title of the paper?,Fast and Secure Laptop Backups with Encrypted ...
84,1002,Title,What is the title of the paper?,What consistency does your key-value store act...
142,1003,Title,What is the title of the paper?,Scalable Web Object Inspection and Malfease Co...
193,1004,Title,What is the title of the paper?,Towards proving security in the presence of la...
266,1005,Title,What is the title of the paper?,Provenance Integration Requires Reconciliation
320,1006,Title,What is the title of the paper?,Detecting Malware Domains at the Upper DNS Hie...
381,1007,Title,What is the title of the paper?,Approximate XML Query Answers
407,1008,Title,What is the title of the paper?,Block-level RAID is dead
469,1009,Title,What is the title of the paper?,Look Who's Talking: Discovering Dependencies b...
528,1010,Title,What is the title of the paper?,On the Benefit of Virtualization: Strategies f...


In [ ]:
df_sampled = question_df.groupby(['paper_id', 'section'], group_keys=False).apply(
    lambda x: x.sample(n=min(3, len(x)), random_state=42)
)

<ipython-input-17-4338a59f22ac>:1: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  df_sampled = question_df.groupby(['paper_id', 'section'], group_keys=False).apply(


In [ ]:
paper_1091 = question_df[question_df['paper_id'] == 1091]

display(paper_1091)

In [ ]:
paper_1087 = question_df[question_df['paper_id'] == 1087]

display(paper_1087)

In [ ]:
uq = question_df['question'].value_counts()

display(uq)

In [ ]:
question_df[question_df['question'] == "What does HiTune demonstrate the benefits of?"]

In [ ]:
vc = question_df['paper_id'].value_counts()

display(vc)

In [ ]:
df_sampled

,paper_id,section,question,answer
1,1001,Abstract:,What type of data does the algorithm take adva...,common
9,1001,Section 1.1: De-duplication & Encryption,What does CAS stand for?,content-addressable storage
8,1001,Section 1.1: De-duplication & Encryption,What is there a good deal of between the data ...,sharing
12,1001,Section 1.2: A Solution,What is the name of the software that overcome...,prototype
19,1001,Section 1.2: A Solution,What is the algorithm capable of identifying i...,shared sub-trees
...,...,...,...,...
6415,1100,Section 6: Related Work,What is the coarse-grained version of informat...,where-provenance
6416,1100,Section 7: Conclusion,What language was the enterprise-grade applica...,Perl
6417,1100,Section 7: Conclusion,What does DBTaint do when database clients ret...,tags
6421,1100,Section 7: Conclusion,What does DBTaint require to the database?,SQL-level additions


In [ ]:
df_sampled.to_csv(os.path.join(base_folder_path, "sampled_question.csv"), index=False)

In [ ]:
sp_vc = df_sampled['paper_id'].value_counts()

display(sp_vc)

,count
paper_id,
1087,102
1076,89
1100,87
1065,83
1070,77
1025,74
1067,74
1092,74
1045,71


In [ ]:
# from transformers import AutoTokenizer, AutoModelForQuestionAnswering
# import torch

# model_name = "deepset/roberta-base-squad2"

# tokenizer = AutoTokenizer.from_pretrained(model_name)
# model = AutoModelForQuestionAnswering.from_pretrained(model_name)

In [ ]:
# def t5_answer_question(question, context, max_length=64): # change to qustion set
#     input_text = f"question: {question}  context: {context}"
#     input_ids = tokenizer.encode(input_text, return_tensors="pt", truncation=True)

#     outputs = model.generate(input_ids, max_length=max_length, early_stopping=True)
#     answer = tokenizer.decode(outputs[0], skip_special_tokens=True)
#     return answer

In [ ]:
# def roberta_answer_question(question, context):
#     inputs = tokenizer.encode_plus(
#         question, context,
#         return_tensors="pt",
#         truncation=True,
#         max_length=512
#     )

#     with torch.no_grad():
#         outputs = model(**inputs)

#     start_scores = outputs.start_logits
#     end_scores = outputs.end_logits

#     # Get the most likely start and end of answer with the argmax
#     start_idx = torch.argmax(start_scores)
#     end_idx = torch.argmax(end_scores) + 1

#     if start_idx >= end_idx:
#         return "No answer found."

#     answer_tokens = inputs["input_ids"][0][start_idx:end_idx]
#     answer = tokenizer.decode(answer_tokens, skip_special_tokens=True)

#     return answer

In [ ]:
client = genai.Client()

response = client.models.generate_content(
  model="gemini-2.0-flash", contents="Explain how AI works in a few words"
)
print(response.text)

AI uses data and algorithms to mimic human intelligence.



In [ ]:
def gemini_answer_question(question, context):
  condition = "If the question asks for the title of the paper, provide the full title with no word limit. For all other questions, limit your answer to a maximum of two words."
  prompt = f"Condition:\n{condition}\nContext:\n{context}\n\nQuestion:\n{question}\n\nAnswer:"
  response = client.models.generate_content(
    model="gemini-2.0-flash", contents=prompt
  )
  return response.text.strip()

In [ ]:
question = "Where do I live?"
context = "My name is Merve and I live in Istanbul."

answer = gemini_answer_question(question, context)
print("Answer:", answer)

Answer: Istanbul


In [ ]:
question = "What is the title of the paper?"
context = test_src

answer = gemini_answer_question(question, context)
print("Answer:", answer)

Answer: Fast and Secure Laptop Backups with Encrypted De-duplication


In [ ]:
question = "What is the title of the paper?"
context = test_sum

answer = gemini_answer_question(question, context)
print("Answer:", answer)

Answer: Fast and Secure Laptop Backups with Encrypted De-duplication


In [ ]:
question = "What is the title of the paper?"
context = test_lora

answer = gemini_answer_question(question, context)
print("Answer:", answer)

Answer: Fast and Secure Laptop Backups with Encrypted De-duplication


In [ ]:
question = "What is the title of the paper?"
context = test_baseline

answer = gemini_answer_question(question, context)
print("Answer:", answer)

Answer: Fast and Secure Laptop Backups with Encrypted De-duplication


In [ ]:
# i = 1001
# answer_from_src_list = []
# answer_from_sum_list = []
# count = 0

# for index in tqdm(range(len(data_df))):
#     paper_id = data_df.loc[index, 'paper_id']
#     src = data_df.loc[index, 'source']
#     sum = data_df.loc[index, 'summary']

#     questions = question_df[question_df['paper_id'] == paper_id]['question'].tolist()

#     # Prepare input batch for source
#     input_texts_src = [f"question: {q}  context: {src}" for q in questions]
#     input_ids_src = tokenizer(input_texts_src, return_tensors="pt", padding=True, truncation=True, max_length=1024)

#     # Prepare input batch for summary
#     input_texts_sum = [f"question: {q}  context: {sum}" for q in questions]
#     input_ids_sum = tokenizer(input_texts_sum, return_tensors="pt", padding=True, truncation=True, max_length=1024)

#     # Generate answers in batch
#     outputs_src = model.generate(**input_ids_src, max_length=64, early_stopping=True)
#     outputs_sum = model.generate(**input_ids_sum, max_length=64, early_stopping=True)

#     answers_src = tokenizer.batch_decode(outputs_src, skip_special_tokens=True)
#     answers_sum = tokenizer.batch_decode(outputs_sum, skip_special_tokens=True)

#     # Store answers
#     for idx, (q, ans_src, ans_sum) in enumerate(zip(questions, answers_src, answers_sum)):
#         answer_from_src_list.append({
#             "q_index": count,
#             "paper_id": paper_id,
#             "question": q,
#             "answer_from_src": ans_src
#         })

#         answer_from_sum_list.append({
#             "q_index": count,
#             "paper_id": paper_id,
#             "question": q,
#             "answer_from_sum": ans_sum
#         })

#         count += 1

#     i += 1

# print("Answers from source:\n", answer_from_src_list)
# print("\nAnswers from summary:\n", answer_from_sum_list)

In [ ]:
# df_answers_src = pd.DataFrame(answer_from_src_list)
# df_answers_sum = pd.DataFrame(answer_from_sum_list)

# df_answers_src.to_csv(os.path.join(base_folder_path,"answers_from_source.csv"), index=False)
# df_answers_sum.to_csv(os.path.join(base_folder_path,"answers_from_summary.csv"), index=False)

In [ ]:
i = 1001
answer_list = []
count = 0

for index in tqdm(range(len(data_df))):
  src = data_df.loc[index, 'source']
  sum = data_df.loc[index, 'summary']
  lor = data_df.loc[index, 'lora']
  bal = data_df.loc[index, 'baseline']

  lim = 0
  for question in tqdm(question_df[question_df['paper_id'] == i]['question']):
    answer_from_src = gemini_answer_question(question, src)
    answer_from_sum = gemini_answer_question(question, sum)
    answer_from_lora = gemini_answer_question(question, lor)
    answer_from_baseline = gemini_answer_question(question, bal)

    answer_list.append({
        "q_index": count,
        "paper_id": i,
        "question": question,
        "section": question_df[(question_df['paper_id'] == i) & (question_df['question'] == question)]['section'].iloc[0],
        "answer_from_src": answer_from_src,
        "answer_from_sum": answer_from_sum,
        "answer_from_lora": answer_from_lora,
        "answer_from_baseline": answer_from_baseline
    })

    if(lim == 30):
      break
    lim += 1
    count += 1

  i += 1
  if(i == 1002):
    break

  0%|          | 0/100 [01:01<?, ?it/s]


In [ ]:
df_answer_list = pd.DataFrame(answer_list)

df_answer_list.to_csv(os.path.join(base_folder_path, "test_answer_list.csv"), index=False)

In [ ]:
i = 1001
answer_sampling_question_list = []
count = 0

for index in tqdm(range(len(data_df))):
  src = data_df.loc[index, 'source']
  sum = data_df.loc[index, 'summary']
  lor = data_df.loc[index, 'lora']
  bal = data_df.loc[index, 'baseline']

  for question in df_sampled[df_sampled['paper_id'] == i]['question']:
    answer_from_src = gemini_answer_question(question, src)
    answer_from_sum = gemini_answer_question(question, sum)
    answer_from_lora = gemini_answer_question(question, lor)
    answer_from_baseline = gemini_answer_question(question, bal)

    section = df_sampled[(df_sampled['paper_id'] == i) & (df_sampled['question'] == question)]['section'].iloc[0]
    actual_answer = df_sampled[(df_sampled['paper_id'] == i) & (df_sampled['question'] == question)]['answer'].iloc[0]

    answer_sampling_question_list.append({
      "q_index": count,
      "paper_id": i,
      "section": section,
      "question": question,
      "actual_answer": actual_answer,
      "answer_from_src": answer_from_src,
      "answer_from_sum": answer_from_sum,
      "answer_from_lora": answer_from_lora,
      "answer_from_baseline": answer_from_baseline
    })
    count += 1

  if (i-1000) % 25 == 0:
    df_answer_sampling_question = pd.DataFrame(answer_sampling_question_list)
    df_answer_sampling_question.to_csv(os.path.join(base_folder_path, f"answer_sampling_checkpoint_{int((i-1000)/25)}_{i-24}-{i}.csv"), index=False)
    answer_sampling_question_list = []

  i += 1

100%|██████████| 100/100 [1:54:10<00:00, 68.51s/it]


In [ ]:
df_answer_sampling_question = pd.DataFrame(answer_sampling_question_list)

df_answer_sampling_question.to_csv(os.path.join(base_folder_path, "answer_sampling_question.csv"), index=False)

In [ ]:
# i = 1076
# answer_sampling_question_list = []
# count = 2705

# for index in tqdm(range(75, len(data_df))):
#   src = data_df.loc[index, 'source']
#   sum = data_df.loc[index, 'summary']
#   lor = data_df.loc[index, 'lora']
#   bal = data_df.loc[index, 'baseline']

#   for question in df_sampled[df_sampled['paper_id'] == i]['question']:
#     answer_from_src = gemini_answer_question(question, src)
#     answer_from_sum = gemini_answer_question(question, sum)
#     answer_from_lora = gemini_answer_question(question, lor)
#     answer_from_baseline = gemini_answer_question(question, bal)

#     section = df_sampled[(df_sampled['paper_id'] == i) & (df_sampled['question'] == question)]['section'].iloc[0]
#     actual_answer = df_sampled[(df_sampled['paper_id'] == i) & (df_sampled['question'] == question)]['answer'].iloc[0]

#     answer_sampling_question_list.append({
#       "q_index": count,
#       "paper_id": i,
#       "section": section,
#       "question": question,
#       "actual_answer": actual_answer,
#       "answer_from_src": answer_from_src,
#       "answer_from_sum": answer_from_sum,
#       "answer_from_lora": answer_from_lora,
#       "answer_from_baseline": answer_from_baseline
#     })
#     count += 1

#   if (i-1000) % 25 == 0:
#     df_answer_sampling_question = pd.DataFrame(answer_sampling_question_list)
#     df_answer_sampling_question.to_csv(os.path.join(base_folder_path, f"answer_sampling_checkpoint_{int((i-1000)/25)}_{i-24}-{i}.csv"), index=False)
#     answer_sampling_question_list = []

#   i += 1